# Demonstration of the ribbon package

When you run this for the first time, you need to install the package, which you can do by uncommenting (i.e., by removing the leading #) the line below

In [8]:
pip uninstall -y ribbon

Found existing installation: ribbon 0.1a0
Uninstalling ribbon-0.1a0:
  Successfully uninstalled ribbon-0.1a0
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install '/Users/ruehle/GitHub/ribbon'

Processing /Users/ruehle/GitHub/ribbon
  Preparing metadata (setup.py) ... done
  Created wheel for ribbon: filename=ribbon-0.1a0-py3-none-any.whl size=352409 sha256=9ed09922de01c4fbb7ad75e28364b6dbb64a8298858854f6a1cfe7c1b990937a
  Stored in directory: /private/var/folders/r3/33sjs05x2w3f5y4cmm3yr7h00000gn/T/pip-ephem-wheel-cache-fsoaqfo7/wheels/3e/6e/d4/c058900cfbbfb8644a09e0d0bea766e2d80bac721088fb525b
Successfully built ribbon
You should consider upgrading via the '/Users/ruehle/testvenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# pip install git+https://github.com/ribbon/ribbon.git

Next, we import the package and a few other ones that will be useful in this demonstration

In [1]:
import ribbon.rw  # the main class
import ribbon.visualizer as visualizer  # for visualizing the output

import numpy as np  # handles arrays
import snappy  # to represent the knots
import logging  # to print some info of what the code is doing

Here are the options we can use to configure the search

In [2]:
links = ['K6a3']          # links to search. Accepts a list of PD codes or link names. K6a3 is the Stevedore knot
max_size = 10             # max number of crossings any intermediate knot can have
max_steps =  5000         # max number of steps searched by the random walker before giving up and resetting the link
max_tries = 10000         # number of total steps before we give up completely
max_bct = 3               # we use the same variable to set the max number of allowed twists, link components, and bands to add. If you start with a knot, the max number of bands is the max number of components + 1, since each band adds a link component
log_level = logging.ERROR # controls how much information is printed by the code while searching for a band
use_checks = False        # If set to true (whcih requires sage), the code will check for slice obstructions after attaching bands, rather than keep searching more and more bands on a link that is potentially obstructed. This uses the Fox Milnor condition, which requires the Alexander Polynomial. This can be slow to compute for large knots
save_images = True        # Will use the visualizer to save the band found by the random walker

In [3]:
random_walker = ribbon.rw.RandomWalker(links=links, 
                                       max_size=max_size, 
                                       max_steps=max_steps, 
                                       max_bct=max_bct, 
                                       logger=None, 
                                       log_level=log_level, 
                                       use_band_checks=use_checks, 
                                       save_solved_knot_images=save_images
                                      )

## Perform random search

Now we perform random actions that lead to random bands until we find a collection of bands that leads to the unknot, or the max number of steps has been exhausted

In [4]:
tries = 0
while tries < max_tries:
    tries += 1
    if tries % 10000 == 0:
        print("Performed {:d} steps.".format(tries))
    
    # Find all valid actions
    valid_actions = np.argwhere(random_walker.invalid_action_mask()).flatten()
    
    # pick a random one
    a = np.random.choice(valid_actions) if len(valid_actions) > 0 else 0
    
    # perform the action
    done, info = random_walker.step(a)
        
    # check whether the knot is done (either because the unknot was reached, or a reset was triggered)
    if done:
        if 'unknot' in info['result']:  # found bands
            print("Knot is ribbon!")  # this is not an error, we just want to print this message irrespective of the logger level.
            break

Knot is ribbon!
Your new Plink window needs an event loop to become visible.
Type "%gui tk" below (without the quotes) to start one.



## Perform random weighted search

Alternatively, we can weight the actions differently

In [6]:
max_action_per_category = max_size + 3
weights = [0.25,0.85,0.05,0.05,0.15]  # this was found to work well
weights = [float(weights[0])] * max_action_per_category + [float(weights[3]), float(weights[1]), float(weights[2])] * max_action_per_category + [float(weights[4])] * 2

In [8]:
# the rest is essentially the same as before
random_walker = ribbon.rw.RandomWalker(links=links, 
                                       max_size=max_size, 
                                       max_steps=max_steps, 
                                       max_bct=max_bct, 
                                       logger=None, 
                                       log_level=log_level, 
                                       use_band_checks=use_checks, 
                                       save_solved_knot_images=save_images
                                      )

tries = 0
while tries < max_tries:
    tries += 1
    if tries % 10000 == 0:
        print("Performed {:d} steps.".format(tries))
    
    # Find all valid actions
    valid_actions = np.array(random_walker.invalid_action_mask(), dtype=float)
    
    # pick a random valid action weighted according to the weights specified above
    ws = weights * valid_actions
    ws = ws / np.sum(ws) if np.sum(ws) != 0 else [1./(4 * max_size + 2)] * (4 * max_size + 2)
    a = np.random.choice(len(valid_actions), p=ws) if valid_actions.any() else 0
    
    # perform the action
    done, info = random_walker.step(a)
        
    # check whether the knot is done (either because the unknot was reached, or a reset was triggered)
    if done:
        if 'unknot' in info['result']:  # found bands
            print("Knot is ribbon!")  # this is not an error, we just want to print this message irrespective of the logger level.
            break

Knot is ribbon!
Your new Plink window needs an event loop to become visible.
Type "%gui tk" below (without the quotes) to start one.



## Perform parallel search

Since the search is random, it can be parallelized very easily, either by running the Random Walker on the same knot multiple times, or over many different knots.

For the parallelization, we use 2 packages that are not necessary to run the program itself:
* joblib to parallelize
* tqdm for a progress bar

If you dont have these packages, you can install them by commenting out the lines below

In [11]:
# pip install joblib tqdm

In [24]:
import joblib  # for parallelization
from tqdm import tqdm  # for progress bar
import contextlib  # for progress bar
import time  # for timing out computation

In [31]:
# this runs the search
# Args:
#    max_size (int):     Max number of crossings of any link encountered
#    timeout (int):      Max time in seconds to search a knot before giving up
#    max_tries (int):    Max number of times a knot is tried before giving up. Set to -1 for infinite tries (or until timeout is reached)
#    max_bands (int):    Max number of bands to try (and max number of twists and max number of components)
#    save_images (bool): Wheter an image that shows the band which was found should be saved
#    use_checks (bool):  Wheter slice obstructions should be checked after each band addition (if True, requires sage)
#    knot_name (str):    A name for the knot (only used for saving the image)
#    weights (list):     A list of weights for the 5 actions
#    max_steps (int):    Max steps to try before resetting the knot and starting over
#    log_level (int):    How much information is printed
#
# Returns:
#    bool:               True if a band was found within the specified time and/or number of tries, false otherwise
def run_rw_parallel(knot, max_size, timeout=5*60, max_tries=-1, max_bands=4, save_images=True, use_checks=False, knot_name="", weights=None, max_steps=50, log_level=logging.ERROR):
    max_bands = max_bands
    num_xing = len(knot)
    max_size = max(int(1.25 * num_xing), max_size)
    max_action_per_category = max_size + 3
    if weights is not None:
        weights = [float(weights[0])] * max_action_per_category + [float(weights[3]), float(weights[1]), float(weights[2])] * max_action_per_category + [float(weights[4])] * 2
    
    if not isinstance(knot, str):
        knot = [[list(k) for k in knot]]
    else:
        knot = [knot]
    
    random_walker = ribbon.rw.RandomWalker(links=knot, 
                                           max_size=max_size, 
                                           max_steps=max_steps, 
                                           max_bct=max_bct, 
                                           log_level=log_level, 
                                           use_band_checks=use_checks, 
                                           save_solved_knot_images=save_images,
                                           save_knot_name=str(knot_name)
                                          )
    start_time, found_bands, tries = time.time(), False, 0
    while (time.time() - start_time < timeout) and (max_tries == -1 or tries <= max_tries):
        try:
            tries += 1
            if tries % 10000 == 0: 
                    print("Performed {:4d} steps. Currently at knot {}.".format(tries, random_walker.L.PD_code()))
            # random sample valid action
            if weights is None:
                valid_actions = np.argwhere(random_walker.invalid_action_mask()).flatten()
                a = np.random.choice(valid_actions) if len(valid_actions) > 0 else 0
            else:
                valid_actions = np.array(random_walker.invalid_action_mask(), dtype=float)
                ws = weights * valid_actions
                ws = ws / np.sum(ws) if np.sum(ws) != 0 else [1. / (4 * max_size + 2)] * (4 * max_size + 2)
                a = np.random.choice(len(valid_actions), p=ws) if valid_actions.any() else 0
            
            done, info = random_walker.step(a)

            if done:
                if 'unknot' in info['result']:  # solved the knot
                    found_bands = True
                    print("Solved knot {}".format(knot_name))
                    break
                else:
                    random_walker.reset("True")
        except Exception as e:
            print("Exception raised:", e)
            random_walker = ribbon.rw.RandomWalker(links=knot, 
                                           max_size=max_size, 
                                           max_steps=max_steps, 
                                           max_bct=max_bct, 
                                           log_level=log_level, 
                                           use_band_checks=use_checks, 
                                           save_solved_knot_images=save_images,
                                           save_knot_name=str(knot_name)
                                          )
    
    return found_bands

# this is for the progress bar
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

run this function multiple times in parallel

In [32]:
knots = ['K13n3162', 'K14a3636', 'K14a5890', 'K14n11236']  # some ribbon knots for which finding bands is hard
weights = [0.25, 0.85, 0.05, 0.05, 0.15]  # these work well
timeout = 5*60  # 60s
n_jobs = 4  # number of parallel search threads

max_size = 40
max_steps =  5000
max_tries = -1  # try until timeout
max_bct = 3
log_level = logging.ERROR
use_checks = False
save_images = True

In [33]:
with tqdm_joblib(tqdm(desc="Testing knots", total=len(knots))) as progress_bar:
    results = joblib.Parallel(n_jobs=n_jobs, backend='loky')(joblib.delayed(run_rw_parallel)(knot, timeout=timeout, save_images=save_images, use_checks=use_checks, max_size=max_size, max_bands=max_bct, weights=weights, max_steps=max_steps, log_level=log_level, knot_name=str(knot)) for knot in knots)


Testing knots: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [05:00<00:00, 75.11s/it]


Performed 10000 steps. Currently at knot [(1, 15, 2, 14), (15, 3, 16, 2), (21, 10, 18, 11), (7, 18, 8, 19), (5, 17, 6, 16), (17, 7, 14, 6), (19, 8, 20, 9), (9, 20, 10, 21), (3, 12, 4, 13), (13, 4, 0, 5), (11, 0, 12, 1)].
Performed 20000 steps. Currently at knot [(1, 5, 2, 4), (5, 3, 0, 2), (7, 10, 8, 11), (11, 8, 6, 9), (9, 6, 10, 7), (3, 1, 4, 0)].
Performed 30000 steps. Currently at knot [(7, 11, 8, 10), (11, 9, 6, 8), (9, 7, 10, 6), (1, 4, 2, 5), (5, 2, 0, 3), (3, 0, 4, 1)].
Solved knot K14n11236


In [37]:
for k, r in zip(knots, results):
    print("Found band for knot {:10s}: {}".format(k, r))

Found band for knot K13n3162  : False
Found band for knot K14a3636  : True
Found band for knot K14a5890  : False
Found band for knot K14n11236 : True
Performed 10000 steps. Currently at knot [(13, 17, 14, 16), (15, 6, 16, 7), (7, 14, 8, 15), (17, 13, 18, 12), (11, 23, 12, 22), (23, 11, 22, 10), (9, 3, 10, 2), (3, 9, 4, 8), (5, 18, 6, 19), (19, 4, 0, 5), (1, 20, 2, 21), (21, 0, 20, 1)].
Performed 20000 steps. Currently at knot [(2, 0, 3, 27), (0, 9, 1, 10), (8, 1, 9, 2), (18, 4, 19, 3), (4, 25, 5, 26), (20, 5, 21, 6), (6, 14, 7, 13), (12, 8, 13, 7), (10, 17, 11, 18), (16, 11, 17, 12), (14, 24, 15, 23), (22, 16, 23, 15), (26, 20, 27, 19), (24, 21, 25, 22)].
Performed 30000 steps. Currently at knot [(2, 0, 3, 27), (0, 9, 1, 10), (8, 1, 9, 2), (18, 4, 19, 3), (4, 25, 5, 26), (20, 5, 21, 6), (6, 14, 7, 13), (12, 8, 13, 7), (10, 17, 11, 18), (16, 11, 17, 12), (14, 24, 15, 23), (22, 16, 23, 15), (26, 20, 27, 19), (24, 21, 25, 22)].
Performed 40000 steps. Currently at knot [(2, 0, 3, 27), (0, 